# 0. Install Dependencies

# 1. Test Random Environment with OpenAI Gym

In [ ]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [ ]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [43]:
env = ShowerEnv()

In [ ]:
env.observation_space.sample()

array([40.04545], dtype=float32)

In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:14
Episode:2 Score:-34
Episode:3 Score:-28
Episode:4 Score:20
Episode:5 Score:-28
Episode:6 Score:26
Episode:7 Score:-14
Episode:8 Score:-20
Episode:9 Score:-52
Episode:10 Score:-36


# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
actions

3

In [ ]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                48        
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [44]:
env.reset()

39

# 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [38]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 16:48 - reward: -1.0000

C:\Users\vansh\AppData\Roaming\Python\Python39\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\vansh\AppData\Roaming\Python\Python39\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 105s 10ms/step - reward: -0.6414
166 episodes - episode_reward: -38.639 [-60.000, 16.000] - loss: 2.132 - mae: 7.668 - mean_q: -5.069

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 183s 18ms/step - reward: -0.5250
167 episodes - episode_reward: -31.377 [-60.000, 36.000] - loss: 1.990 - mae: 9.451 - mean_q: -13.464

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 142s 14ms/step - reward: -0.5224
167 episodes - episode_reward: -31.341 [-60.000, 32.000] - loss: 2.714 - mae: 11.985 - mean_q: -17.279

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 166s 17ms/step - reward: -0.5088
166 episodes - episode_reward: -30.422 [-60.000, 34.000] - loss: 2.633 - mae: 11.766 - mean_q: -16.946

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 99s 10ms/step - reward: -0.5066
done, took 695.584 seconds


In [39]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: -60.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: -60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: -60.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: -60.000, steps: 60
Episode 17: reward: -60.000, steps: 60
Episode 18: reward: -60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: -60.000, steps: 60
Episode 21: reward: -60.000, steps: 60
Episode 22: reward: -60.000, steps: 60
Episode 23: reward: -60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: rewa

# 4. Reloading Agent from Memory

In [40]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [41]:
del model
del dqn
del env

In [42]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

NameError: name 'gym' is not defined

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
